In [22]:
from pyspark.sql import SparkSession
import random
import sys

In [7]:
import os

In [8]:
os.cpu_count()

2

In [ ]:
# Create a long int list 
int_seq = []

for i in range(10000):
    int_seq.append(random.randint(1, 100000))

In [ ]:
int_seq[:20]

In [20]:
#create a spark session
spark = SparkSession.builder.appName("mapPartitions").getOrCreate()
sc = spark.sparkContext
#specify number of partitions
rddObj = sc.parallelize(int_seq, numSlices=20)

#show number of partitions
rddObj.getNumPartitions()

20

In [30]:
#have such levels : ALL, DEBUG, ERROR, FATAL, INFO, OFF, TRACE, WARN
sc.setLogLevel("Warn")

In [21]:
#let us see number of executors and number of cores of executors
from pyspark import SparkConf

SparkConf().getAll()

[('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'mapPartitions')]

In [19]:
spark.conf

In [18]:
sc

<SparkContext master=local[*] appName=mapPartitions>

In [17]:
print(sc.getConf().get("sprak.executor.instances"))
print(sc.getConf().get("sprak.executor.cores"))

None
None


In [24]:
sys.maxsize

9223372036854775807

In [34]:
#Next we will caculate min max count of each partitions
def min_max_count(partition):
    maxNum = 0
    minNum = sys.maxsize
    count = 0
    for element in partition:
        if(element > maxNum):
            maxNum = element
        if(element < minNum):
            minNum = element
        count +=1

    return [(maxNum, minNum, count)]

In [35]:
mapped_rdd = rddObj.mapPartitions(min_max_count)

In [36]:
mapped_rdd.collect()

[(99833, 330, 500),
 (99801, 93, 500),
 (99982, 20, 500),
 (99888, 555, 500),
 (99859, 574, 500),
 (99785, 204, 500),
 (99894, 282, 500),
 (99966, 15, 500),
 (99733, 1302, 500),
 (99913, 100, 500),
 (99513, 34, 500),
 (99857, 465, 500),
 (99924, 274, 500),
 (99771, 171, 500),
 (99916, 294, 500),
 (99399, 57, 500),
 (99947, 321, 500),
 (99870, 122, 500),
 (99977, 287, 500),
 (99794, 815, 500)]

In [37]:
#use reduce to calculate the min max count of all the partitions
reduced_rdd = mapped_rdd.reduce(lambda a, b: (max(a[0], b[0]), min(a[1], b[1]), a[2] + b[2]))

In [39]:
reduced_rdd

(99982, 15, 10000)

In [40]:
spark.stop()